In [1]:
import torch
import numpy as np
import sys

sys.path.append('human_body_prior/tools/')
from omni_tools import copy2cpu as c2c
from os import path as osp

motionx_dir = 'data/motion-x/'
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(comp_device)

cuda


In [2]:
sys.path.append('human_body_prior/body_model/')
from body_model import BodyModel

gender = "male"

bm_fname = 'smplh_models/{}/model.npz'.format(gender)
bm = BodyModel(bm_fname=bm_fname).to(comp_device)
faces = c2c(bm.f)


In [3]:
motionx_npy = osp.join(motionx_dir,'smplx','HAA500','subset_0008','Gangnam_Style_Dance_clip_12.npy')
mdata = np.load(motionx_npy)

len = mdata.shape[0]
print("Motion data shape: {}\n".format(mdata.shape))

motion = torch.tensor(mdata).float()
motion_parms = {
            'root_orient': motion[:, :3].to(comp_device),  # controls the global root orientation
            'pose_body': motion[:, 3:3+63].to(comp_device),  # controls the body
            'pose_hand': motion[:, 66:66+90].to(comp_device),  # controls the finger articulation
            'pose_jaw': motion[:, 66+90:66+93].to(comp_device),  # controls the yaw pose
            'face_expr': motion[:, 159:159+50].to(comp_device),  # controls the face expression
            'face_shape': motion[:, 209:209+100].to(comp_device),  # controls the face shape
            'trans': motion[:, 309:309+3].to(comp_device),  # controls the global body position
            'betas': motion[:, 312:].to(comp_device),  # controls the body shape. Body shape is static
        }

print('Body parameter vector shapes: \n{}'.format(' \n'.join(['{}: {}'.format(k,v.shape) for k,v in motion_parms.items()])))

Motion data shape: (193, 322)

Body parameter vector shapes: 
root_orient: torch.Size([193, 3]) 
pose_body: torch.Size([193, 63]) 
pose_hand: torch.Size([193, 90]) 
pose_jaw: torch.Size([193, 3]) 
face_expr: torch.Size([193, 50]) 
face_shape: torch.Size([193, 100]) 
trans: torch.Size([193, 3]) 
betas: torch.Size([193, 10])


In [4]:
import trimesh
sys.path.append('body_visualizer')
from tools.vis_tools import colors
from mesh.mesh_viewer import MeshViewer
from mesh.sphere import points_to_spheres
from tools.vis_tools import show_image

imw, imh=1600, 1600
mv = MeshViewer(width=imw, height=imh, use_offscreen=True)

In [5]:
body_pose_beta = bm(**{k:v for k,v in motion_parms.items() if k in ['pose_body', 'pose_hand', 'root_orient', 'betas']})

def vis_body_hand_pose(fId = 0, SHOW=True):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_pose_beta.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    if SHOW: show_image(body_image)

    return body_image

body_images = []
for i in range(len):
    body_images.append(vis_body_hand_pose(fId=i, SHOW=False))

In [6]:
import mediapy as media

media.show_video(images=body_images,fps=30,width=700)

This browser does not support the video tag.
